# Dashboard Feeds

In [7]:
import io
import sys
import numpy as np
import pandas as pd
from collections import OrderedDict
import xlrd
import IPython
import bokeh
from bokeh.io import output_notebook, output_file, show, push_notebook
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Tabs, Panel
from bokeh.transform import factor_cmap
import hvplot as hv
import hvplot.pandas
from simple_salesforce import Salesforce
from pardot.client import APIClient
import pytz
import datetime
from progressbar import ProgressBar
print("python:", sys.version)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("xlrd:", xlrd.__version__)
print("IPython:", IPython.__version__)
print("bokeh:", bokeh.__version__)
print("holoviews:", hv.__version__)
print("Salesforce:", Salesforce)
print("Pardot:", APIClient)
output_notebook()
from simple_salesforce import Salesforce
from salesforce_reporting import Connection, ReportParser
with open('.sfdc') as f:
    uname, spass, stoken, ppass, ptoken = [x.strip("\n") for x in f.readlines()]

python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
numpy: 1.16.3
pandas: 0.24.2
xlrd: 1.2.0
IPython: 7.1.1
bokeh: 1.1.0
holoviews: 0.4.0
Salesforce: <class 'simple_salesforce.api.Salesforce'>
Pardot: <class 'pardot.client.APIClient'>


Loading BokehJS ...

In [10]:
sf = Salesforce(username=uname, password=spass, security_token=stoken)   # authenticate
end = datetime.datetime.now(pytz.UTC)                     # salesforce API requires UTC
# print("Session ID:", session_id)
# print("Instance:", sf_instance)
# print("Version:", sf_version)
print("Authenticated @", end)

Authenticated @ 2019-06-16 04:29:26.962108+00:00


In [ ]:
def executeScriptsFromFile(filename):
    fd = open('ZooDatabase.sql', 'r')   # Open and read file as a single buffer
    soql = fd.read()
    fd.close()
    soqlCmds = soql.split(';')          # all SOQL commands (split on ';')
    for command in soqlCmds:            # Execute every command from the input file
        try:                            # This will skip and report errors
            c.execute(command)
        except OperationalError, msg:
            print "Command skipped: ", msg

In [ ]:
executeScriptsFromFile('zookeeper.sql')
import sqlite3
from sqlite3 import OperationalError

conn = sqlite3.connect('csc455_HW3.db')
c = conn.cursor()

# Open and read the file as a single buffer
fd = open('ZooDatabase.sql', 'r')
sqlFile = fd.read()
fd.close()

# all SQL commands (split on ';')
sqlCommands = sqlFile.split(';')

# Execute every command from the input file
for command in sqlCommands:
    # This will skip and report errors
    # For example, if the tables do not yet exist, this will skip over
    # the DROP TABLE commands
    try:
        c.execute(command)
    except OperationalError, msg:
        print "Command skipped: ", msg


# For each of the 3 tables, query the database and print the contents
for table in ['ZooKeeper', 'Animal', 'Handles']:


    **# Plug in the name of the table into SELECT * query
    result = c.execute("SELECT * FROM %s;" % table);**

    # Get all rows.
    rows = result.fetchall();

    # \n represents an end-of-line
    print "\n--- TABLE ", table, "\n"

    # This will print the name of the columns, padding each name up
    # to 22 characters. Note that comma at the end prevents new lines
    for desc in result.description:
        print desc[0].rjust(22, ' '),

    # End the line with column names
    print ""
    for row in rows:
        for value in row:
            # Print each value, padding it up with ' ' to 22 characters on the right
            print str(value).rjust(22, ' '),
        # End the values from the row
        print ""

c.close()
conn.close()

In [26]:
leads = "SELECT mkt_bucket__c, Company, Title, Lead_Type__c, Status FROM Lead WHERE Industry_Vertical__c != NULL AND mkt_bucket__c in ('Paid', 'Organic', 'Email', 'Event', 'Call') AND Lead_Type__c in ('Lead', 'MQL', 'SQL') ORDER BY CreatedDate DESC NULLS LAST LIMIT 5"

In [ ]:
optys = "SELECT Name, mkt_bucket__c, Industry_Vertical__c FROM Opportunity WHERE mkt_bucket__c in ('Paid', 'Organic', 'Email', 'Event', 'Call') ORDER BY CreatedDate DESC NULLS LAST LIMIT 5"

In [ ]:
won = "SELECT Name, mkt_bucket__c, Account.Industry_Vertical__c, TNS_ARR__c FROM Opportunity WHERE mkt_bucket__c in ('Paid', 'Organic', 'Email', 'Event', 'Call') AND Account.Industry_Vertical__c != 'Partner' AND StageName = 'Closed - Signed' AND TNS_ARR__c >= 1000 ORDER BY CloseDate DESC NULLS LAST LIMIT 5"

In [28]:
# query = sf.query(opty)
# query = sf.query_all(opty)
# df = pd.DataFrame(query['records'])
# df = pd.DataFrame(sf.query_all(q_cloud_t3)['records'])
query = sf.query(leads)

In [29]:
query

OrderedDict([('totalSize', 5),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Lead'),
                                          ('url',
                                           '/services/data/v38.0/sobjects/Lead/00Q2H00002FytQDUAZ')])),
                            ('mkt_bucket__c', 'ORGANIC'),
                            ('Company', 'Coury Hospitality'),
                            ('Title', 'Director of Quality Assurance'),
                            ('Lead_Type__c', 'Lead'),
                            ('Status', 'Attempted to Contact')]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Lead'),
                                          ('url',
                                           '/services/data/v38.0/sobjects/Lead/00Q2H00002Fysx2UAB')])),
                            ('mkt_bucket__c', 'CALL'),
                            ('Comp

In [13]:
records = [dict(
      Create=rec['CreatedDate'],
        Name=rec['Name'],
      bucket=rec['mkt_bucket__c'])
    for rec in query['records']]
qdf = pd.DataFrame(records)

In [14]:
qdf

,Create,Name,bucket
0,2009-09-22T15:05:53.000+0000,Citi Performing Arts Center,ORGANIC
1,2011-06-14T20:54:37.000+0000,Markoja d.o.o.-PDO,ORGANIC
2,2009-09-29T14:54:54.000+0000,"Woods Construction, Inc",ORGANIC
3,2011-06-06T18:05:01.000+0000,Adult and Pediatric Urology- New Web Lead,None
4,2009-09-18T16:54:23.000+0000,Dreamlike Pictures,SALES


# Pandas Functions

```python
# read in
df = pd.read_csv('data/hosp.csv', index_col='Target?', dtype={'user_id': int})
df = pd.read_csv('data/15Pct.csv')
with open('csvfile.csv', 'w') as csvfile: f = csv.writer(csvfile) f.writerows(items)  # write to csv file

# dataframes
DataFrame.shape                             # dimensions (tuple)
DataFrame.describe()                        # summary
DataFrame.info()                            # types
DataFrame.dtypes                            # column labels & types
DataFrame.index                             # index (row labels)
DataFrame.head(n)                           # return first n rows
DataFrame.tail(n)                           # return last n rows    
DataFrame.columns                           # column labels
DataFrame.size                              # number (int) elements
DataFrame.memory_usage([index, deep])       # each column memory (bytes)  

# plots
DataFrame.plot([x, y, kind, ax, ….])        # plotting accessor and method
DataFrame.plot.area([x, y])                 # stacked area plot
DataFrame.plot.bar([x, y])                  # vertical bar plot
DataFrame.plot.barh([x, y])                 # horizontal bar plot
DataFrame.plot.box([by])                    # box plot
DataFrame.boxplot([column, by, ax, …])      # columns box plot
DataFrame.plot.density([bw_method, ind])    # Kernel Density Estimate plot using Gaussian kernels
DataFrame.plot.hexbin(x, y[, C, …])         # hexagonal binning plot
DataFrame.plot.hist([by, bins])             # Draw one histogram of the DataFrame’s columns
DataFrame.plot.line([x, y])                 # columns as lines
DataFrame.plot.pie([y])                     # pie plot
DataFrame.plot.scatter(x, y[, s, c])        # scatter plot with varying marker point sizes & colors
DataFrame.hist([column, by, grid, …])       # histogram

# transforms
DataFrame.isna()	                            # Detect missing values.
DataFrame.between_time(start_time, end_time)	# Select between times of the day (e.g., 9:00-9:30 AM)
DataFrame.drop([labels, axis, index, …])	    # Drop specified labels from rows or columns
DataFrame.set_index(keys[, drop, append, …])	# Set the DataFrame index using existing columns
DataFrame.['col0'].astype(str)                  # change dtype (int32, int64, str, bool, category
DataFrame = DataFrame.set_index("col0")
DataFrame.drop("Ireland", axis=0)               # Delete all rows with label "Ireland"
   
# sort
DataFrame.sort_index(axis=2, ascending=False)                 # axis sort
DataFrame.sort_values(by='Target?')                           # value sort 'column'
map() method produces complex criteria
criterion = DataFrame['a'].map(lambda x: x.startswith('t'))   # only want 'two' or 'three'
DataFrame[criterion]
DataFrame[[x.startswith('t') for x in DataFrame['a']]]        # equivalent but slower
DataFrame[criterion & (DataFrame['b'] == 'x')]                # with multiple criteria
DataFrame.loc[criterion & (DataFrame['b'] == 'x'), 'b':'c']   # Select by Label, Position, Advanced Indexing

# dupe handling
DataFrame.drop_duplicates('a', keep='last')
# + keep='first' (default): mark / drop duplicates except for the first occurrence
# + keep='last': mark / drop duplicates except for the last occurrence
# + keep=False: mark / drop all duplicates
DataFrame.index.duplicated()
DataFrame[~DataFrame.index.duplicated()]
DataFrame[~DataFrame.index.duplicated(keep=False)]
DataFrame.loc[['Andrade', 'Veness'], 'city':'email']                    # query 
DataFrame.loc['Andrade':'Veness', ['first_name', 'address', 'city']]    # filter 
DataFrame.loc[DataFrame['first_name'] == 'Antonio', 'city':'email']
DataFrame.loc[DataFrame['email'].str.endswith("hotmail.com")]

# excel
pip install xlrd
import xlrd
pd.read_excel('examples/ex1.xlsx', 'Sheet1')
pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save() 
```

## [Salesforce APIs](https://developer.salesforce.com/docs/api-explorer)    

Create file /root/.sfdc with credentials as:  
SF/Pardot username<cr>  
SF password<cr>  
SF token<cr>  
Pardot Password<cr>  
Pardot token<cr>  

```python
# authenticate 
from salesforce_reporting import Connection, ReportParser
with open('.sfdc') as f:
    username, password, token, uid, instance = [x.strip("\n") for x in f.readlines()]
sf = Salesforce(username=username, password=password, security_token=token, sandbox=False)

# SOQL query examples
ContactDict = sf.query("SELECT Company, CreatedDate, Email, FirstName, Industry, LastName, LeadSource, Title FROM Lead ORDER BY CreatedDate DESC NULLS FIRST LIMIT 50")

opportunity_amounts = sf.query("SELECT Id, Probability, StageName, Amount, Owner.Name FROM Opportunity WHERE AMOUNT < 10000")

amounts = [x['Amount'] for x in opportunity_amounts['records']]
owners = [x['Owner']['Name'] for x in opportunity_amounts['records']]
hist1 = go.Histogram(x=amounts)
py.iplot([hist1], filename='salesforce/opportunity-probability-histogram')

sf.Contact.updated(end - datetime.timedelta(days=10), end)       # contacts updated last 10d
contact = sf.Contact.get('003e0000003GuNXAA0')                   # 18 digit id
metadata = sf.Contact.metadata()
describe = sf.Contact.describe()

# read queryinto dataframe
df = pd.DataFrame(ContactDict['records'])
df.head(10)
Age = df['Age__c']
Age.head(10)
agePlot = Age.hist(bins=10)

# Reports API https://github.com/cghall/force-retrieve
pip install salesforce-reporting
report = sf.get_report('report_id', includeDetails=True)
parser = salesforce_reporting.ReportParser(report)
parser.records()
report = my_sf.get_report('report_id')
matrix_parser = salesforce_reporting.MatrixParser(report)
matrix_parser.series_down('Jan 2016')

# Streaming Client built on salesforce-requests-oauthlib and python-bayeux
pip3 install salesforce-streaming-client
# To test install py.test and pytest-cov in virtualenv
py.test --cov=src/salesforce_streaming_client/ --cov-report html:coverage  # results in ./coverage

# Pardot v3 API for non-dupe email configs https://github.com/joshgeller/PyPardot
pip install pypardot
from pypardot.client import PardotAPI

with open(.pardot) as f:
    username, password, token = [x.strip("\n") for x in f.readlines()]
pa = PardotAPI(email=username, password=password, user_key=token)
pa.authenticate()

pa.prospects.query(created_after='yesterday')

# Pardot API v3 alternative https://github.com/maxnaude/pardot-api-client
pip install pardot-api-client
from pardot.client import APIClient
with open('.pardot') as f:
    username, password, token = [x.strip("\n") for x in f.readlines()]
client = APIClient(username, password, token)
pa = client.prospect.read('someone@example.com')
pa = client.prospect.update('someone@example.com', first_name='John')
for r in client.prospect.query(new=True): print(r)
    
# Pardot API v4 for duped email configs https://github.com/mneedham91/PyPardot4
pip install pypardot4
pa = PardotAPI(email='email',password='password',user_key='user_key')              # Authenticate
pa.authenticate()
pa.prospects.create(email='joe@company.com', first_name='Joe', last_name='Schmoe') # Create new prospect
pa.prospects.create_by_email(email='joe@company.com', first_name='Joe', last_name='Schmoe')
pa.prospects.read_by_email(email='joe@company.com'))                               # query a prospect
prospects = pa.prospects.query(created_after='yesterday')
total = prospects['total_results']                                                 # number of matching records
for prospect in prospects['prospect']
  print(prospect.get('first_name'))
pa.emails.send_to_email(prospect_email='joe@company.com', email_template_id=123)
```